In [1]:
## Importing packages

import numpy as np
import pandas as pd
from zipfile import ZipFile
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
import dill

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, f1_score, confusion_matrix, plot_roc_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_predict
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

from patsy import dmatrices, dmatrix, build_design_matrices

# Set number of CPU cores for parallel algorithms
import os
if "CPU_LIMIT" in os.environ:
    # If you are on JupyterHub, this gives you the right number of CPUs for your virtual machine
    num_cpus = int(os.getenv("CPU_LIMIT").split('.')[0])
else:
    # If you are not on JupyterHub, this gives you the right number for your computer.
    num_cpus = os.cpu_count()
    
# This makes it so that the pandas dataframes don't get truncated horizontally.
pd.options.display.max_columns = 200

In [2]:
## NOTE: taken from class starter code
def summarize(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.DataFrame({'Variable Name': df.columns,
                                   'Data Type': df.dtypes,
                                   'Missing Values': df.isnull().sum(),
                                   'Unique Values': [df[name].nunique() for name in df.columns]}
                                 ).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))

In [3]:
## NOTE: taken from class starter code

from sklearn.base import BaseEstimator, TransformerMixin

class CategoricalImputer(BaseEstimator, TransformerMixin):
    """
    Custom defined imputer for categorical data. This allows you to specify an 
    other class where any category that doesn't meet the requirements necessary to
    be in 
    """
    
    def __init__(self, other_threshold=0.01, 
                 other_label="OTHER",
                 missing_first=True,
                 missing_values=np.nan, 
                 strategy='constant', 
                 fill_value="MISSING", 
                 verbose=0, 
                 copy=True, 
                 add_indicator=False):
        self.add_indicator = add_indicator
        self.copy=copy
        self.verbose=verbose
        self.fill_value=fill_value
        self.missing_first=missing_first
        self.missing_values=missing_values
        self.other_label=other_label
        self.other_threshold=other_threshold
        self.strategy=strategy
        if hasattr(missing_values, "__iter__"):
            self.missing_values = missing_values
        else:
            self.missing_values = [missing_values]
        self._imputer = SimpleImputer(missing_values=missing_values, strategy=strategy, fill_value=fill_value, verbose=verbose, copy=copy, add_indicator=False)
        self._column_categories = {}

        
    def fit(self, X, y=None):
        if type(self.other_threshold) == int or type(self.other_threshold) == float:
            other_threshold = [self.other_threshold]*len(X.columns)
        elif len(self.other_threshold) == len(X.columns):
            other_threshold = self.other_threshold
        else:
            raise TypeError("other_threshold must be either a single number or a list of numbers equal to the number of columns.")

        i = 0
        X = X.copy()
        X = X[:].astype(object)
        if self.missing_first:
            X = pd.DataFrame(self._imputer.fit_transform(X), columns=X.columns, index=X.index)
        column_categories = {}
        for column in X.columns:
            if other_threshold[i] < 1:
                other_threshold[i] = other_threshold[i]*X[column].shape[0]
            
            value_counts = X[column].value_counts()
            categories = [category for category in value_counts.index if value_counts.loc[category] >= other_threshold[i]]
            if value_counts.iloc[-1] >= other_threshold[i]:
                categories[-1] = self.other_label
            else:
                categories.append(self.other_label)
            
            self._column_categories[column] = categories
            i = i + 1
        
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        X = X[:].astype(object)
        if self.missing_first:
            X = pd.DataFrame(self._imputer.fit_transform(X), columns=X.columns, index=X.index)
        for column in X.columns:
            X.loc[~X[column].isin(self._column_categories[column]) & ~X[column].isin(self.missing_values), column] = self.other_label
        return pd.DataFrame(self._imputer.fit_transform(X), columns=X.columns, index=X.index)[:].astype(str)

## Data Cleaning

In [2]:
## Download data

#zf = ZipFile('train.csv.zip')
#zf.extractall()
#zf.close()
df_train=pd.read_csv("train 2.csv")


/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_train.shape

(2026735, 83)

In [5]:
## Exploratory

df_train.head()
df_train.columns
df_train.shape # 544043 rows, 83 columns
df_train.columns
df_train.PuaMode.value_counts()
max(df_train.MachineIdentifier.value_counts()) ## Good, so each row is an individual machine. This is not time series.

1

### 1. NA Column Removal

In [6]:
## Data dictionary: https://www.kaggle.com/c/microsoft-malware-prediction/data?select=test.csv
## According to data dictionary, there are several columns that are entirely NA. I'm going to drop them to avoid confusion.

removals=["RtpStateBitfield","IsSxsPassiveMode","AVProductsInstalled", "AVProductsEnabled", "IeVerIdentifier",                     
                        "Census_OEMNameIdentifier","Census_OEMModelIdentifier","Census_ProcessorManufacturerIdentifier",
                        "Census_ProcessorModelIdentifier","Census_InternalBatteryType","Census_InternalBatteryNumberOfCharges",
                        "Census_OSInstallLanguageIdentifier","Census_OSUILocaleIdentifier","Census_IsFlightingInternal",
                        "Census_ThresholdOptIn","Census_FirmwareManufacturerIdentifier","Census_FirmwareVersionIdentifier",
                        "Census_IsWIMBootEnabled","Wdft_RegionIdentifier"]
len(removals) # There are 19 categories, so dataset should be 19 features smaller. 83-19=64
df_train.drop(removals, axis=1, inplace=True)
df_train.shape # Successfully dropped

(2026735, 64)

### 2. Confusing Feature Name Change

In [7]:
## Change confusing columns. Prefixed Defender info with "Def". Description of some specific variables below.

'''
AutoSampleOptIn: I'm interpreting this to mean opting in to Windows reports when error occurs on machine

PuaMode: Whether "Potentially Unwanted Applications" setting is turned on. This warns when you're downloading something sketch.
            Currently this is either NaN or "on." Not sure if data is missing or NaNs are "off".
            
SMode: Whether setting is enabled to only allow Microsoft Store apps to be installed

SmartScreen: another anti-phishing setting

Firewall: binary of whether Firewall is enabled. Only reports 1 for Windows8.1+, so could be many misleading rows.

UacLuaenable -> AdminMode: setting that requires admin approval for sensitive actions


'''
new_names={"ProductName":"Def_Name", "EngineVersion":"Def_EngineVersion", "AppVersion":"Def_AppVersion",
           "AvSigVersion":"Def_AvSigVersion","IsBeta":"Def_IsBeta", "AVProductStatesIdentifier":"Def_Configuration",
           "HasTpm":"Has_Tpm_Chip", "UacLuaenable":"AdminMode","Census_MDC2FormFactor":"Device_Type",
           "Census_DeviceFamily":"DeviceFamily","Census_ProcessorCoreCount":"ProcessorCoreCount",
           "Census_ProcessorClass":"ProcessorClass", "Census_PrimaryDiskTotalCapacity":"PrimaryDiskTotalCapacity",
           "Census_PrimaryDiskTypeName":"PrimaryDiskTypeName","Census_SystemVolumeTotalCapacity":"SystemVolumeTotalCapacity",
           "Census_HasOpticalDiskDrive":"HasOpticalDiskDrive"}
df_train.rename(columns=new_names,inplace=True)

In [8]:
## Loop for removing "Census" from those feature names

empty=list()
for column in df_train.columns:
    temp=column.replace("Census_","")
    empty.append(temp)
df_train.columns=empty
len(df_train.columns) ## Good - still has same number of features

64

### 3. Imputing Values and Cat./Cont. Variables

In [9]:
## Create lists of categorical and numerical variables. NOTE: numerical will include binary, int, float dtypes

cat_variables=list()
cont_variables=list()

temp=pd.DataFrame(df_train.dtypes)
for column in df_train.columns:
    if(df_train[column].dtypes=="object"):
        cat_variables.append(column)
    else:
        cont_variables.append(column)
print(cat_variables)
print(cont_variables)

['MachineIdentifier', 'Def_Name', 'Def_EngineVersion', 'Def_AppVersion', 'Def_AvSigVersion', 'Platform', 'Processor', 'OsVer', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition', 'PuaMode', 'SmartScreen', 'Device_Type', 'DeviceFamily', 'ProcessorClass', 'PrimaryDiskTypeName', 'ChassisTypeName', 'PowerPlatformRoleName', 'OSVersion', 'OSArchitecture', 'OSBranch', 'OSEdition', 'OSSkuName', 'OSInstallTypeName', 'OSWUAutoUpdateOptionsName', 'GenuineStateName', 'ActivationChannel', 'FlightRing']
['Def_IsBeta', 'DefaultBrowsersIdentifier', 'Def_Configuration', 'Has_Tpm_Chip', 'CountryIdentifier', 'CityIdentifier', 'OrganizationIdentifier', 'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'OsBuild', 'OsSuite', 'IsProtected', 'AutoSampleOptIn', 'SMode', 'Firewall', 'AdminMode', 'ProcessorCoreCount', 'PrimaryDiskTotalCapacity', 'SystemVolumeTotalCapacity', 'HasOpticalDiskDrive', 'TotalPhysicalRAM', 'InternalPrimaryDiagonalDisplaySizeInInches', 'InternalPrimaryDisplayResolutionHorizontal', 'In

In [10]:
## Want to see a summary of categorical variables

summarize(df_train[cat_variables])

'''

--- Notes about our categorical variables, just problematic variables ---

*Drop OSVer. Census_OSVersion codes for the same thing in more detail*
*Make version variables continuous by removing the period*


PuaMode has a TON of missing values, and no "off" values. Therefore, I think it's safe to assume all NaNs are "None."

OsBuildLab only has 4 missing values. Classify as "Other."

SmartScreen is a mess. Merged the "On" and "Off" strings. Unsure about 700K+ missing values. Classify as "Other."

ProcessorClass has low, mid, high rating. It has 2M+ missing values. I'm going to vote for removing it, since we have other
variables accounting for process.

PrimaryDiskTypeName: Merge "unspecified' and "unknown" category as "Other". Classify NaN as "Other"

ChassisTypeName: Collapse "UNKNOWN", "Other", "Unknown" as "Other". Classify NaN as "Other."

PowerPlatformRoleName: Collapse "UNKNOWN", "Unspecified" into "Other". Classify NaN as "Other"

FlightRing: Classify Nan as "Other."

'''


,Data Type,Missing Values,Unique Values,count,unique,top,freq
MachineIdentifier,object,0,2026735,2026735,2026735,0000028988387b115f69f31a3bf04f09,1
Def_Name,object,0,6,2026735,6,win8defender,2005251
Def_EngineVersion,object,0,64,2026735,64,1.1.15200.1,874116
Def_AppVersion,object,0,99,2026735,99,4.18.1807.18075,1168238
Def_AvSigVersion,object,0,7979,2026735,7979,1.273.1420.0,23406
Platform,object,0,4,2026735,4,windows10,1957884
Processor,object,0,3,2026735,3,x64,1841383
OsVer,object,0,33,2026735,33,10.0.0.0,1961065
OsPlatformSubRelease,object,0,9,2026735,9,rs4,889192
OsBuildLab,object,4,546,2026731,546,17134.1.amd64fre.rs4_release.180410-1804,830866


'\n\n--- Notes about our categorical variables, just problematic variables ---\n\n*Drop OSVer. Census_OSVersion codes for the same thing in more detail*\n*Make version variables continuous by removing the period*\n\n\nPuaMode has a TON of missing values, and no "off" values. Therefore, I think it\'s safe to assume all NaNs are "None."\n\nOsBuildLab only has 4 missing values. Classify as "Other."\n\nSmartScreen is a mess. Merged the "On" and "Off" strings. Unsure about 700K+ missing values. Classify as "Other."\n\nProcessorClass has low, mid, high rating. It has 2M+ missing values. I\'m going to vote for removing it, since we have other\nvariables accounting for process.\n\nPrimaryDiskTypeName: Merge "unspecified\' and "unknown" category as "Other". Classify NaN as "Other"\n\nChassisTypeName: Collapse "UNKNOWN", "Other", "Unknown" as "Other". Classify NaN as "Other."\n\nPowerPlatformRoleName: Collapse "UNKNOWN", "Unspecified" into "Other". Classify NaN as "Other"\n\nFlightRing: Classify

In [11]:
#df_train.FlightRing.value_counts()

#### a. Redundant Label Merging - Categorical Variables

In [12]:
## SmartScreen

df_train.SmartScreen.value_counts()

RequireAdmin    980535
ExistsNotSet    237285
Off              42715
Warn             30604
Prompt            7869
Block             5158
off                325
On                 160
&#x02;              85
&#x01;              71
on                  32
Promt                2
0                    2
requireadmin         2
OFF                  1
Name: SmartScreen, dtype: int64

In [13]:
df_train.loc[(df_train.SmartScreen=="OFF") | (df_train.SmartScreen=="off") | (df_train.SmartScreen=="0"), "SmartScreen"] = "Off"
df_train.loc[(df_train.SmartScreen=="on"), "SmartScreen"] = "On"

In [14]:
## PrimaryDiskTypeName

df_train.PrimaryDiskTypeName.value_counts()

HDD            1318412
SSD             560974
UNKNOWN          81201
Unspecified      63211
Name: PrimaryDiskTypeName, dtype: int64

In [15]:
df_train.loc[(df_train.PrimaryDiskTypeName=="UNKNOWN") | (df_train.PrimaryDiskTypeName=="Unspecified"), "PrimaryDiskTypeName"] = "Other"
df_train.PrimaryDiskTypeName.value_counts()

HDD      1318412
SSD       560974
Other     144412
Name: PrimaryDiskTypeName, dtype: int64

In [16]:
## ChassisTypeName

df_train.ChassisTypeName.value_counts()

Notebook               1191862
Desktop                 425895
Laptop                  155675
Portable                 82277
AllinOne                 46632
MiniTower                19318
Convertible              19058
Other                    17245
UNKNOWN                  15207
Detachable               11714
LowProfileDesktop        11326
HandHeld                 10347
SpaceSaving               6475
Tablet                    3053
Tower                     2879
Unknown                   2301
MainServerChassis         2108
MiniPC                    1006
LunchBox                   883
RackMountChassis           781
SubNotebook                177
BusExpansionChassis        169
30                          51
0                           38
StickPC                     34
MultisystemChassis          18
Blade                       17
SealedCasePC                11
35                          10
PizzaBox                     8
31                           3
88                           3
Expansio

In [17]:
df_train.loc[(df_train.ChassisTypeName=="UNKNOWN") | (df_train.ChassisTypeName=="Unknown"), "ChassisTypeName"] = "Other"
df_train.ChassisTypeName.value_counts()

Notebook               1191862
Desktop                 425895
Laptop                  155675
Portable                 82277
AllinOne                 46632
Other                    34753
MiniTower                19318
Convertible              19058
Detachable               11714
LowProfileDesktop        11326
HandHeld                 10347
SpaceSaving               6475
Tablet                    3053
Tower                     2879
MainServerChassis         2108
MiniPC                    1006
LunchBox                   883
RackMountChassis           781
SubNotebook                177
BusExpansionChassis        169
30                          51
0                           38
StickPC                     34
MultisystemChassis          18
Blade                       17
SealedCasePC                11
35                          10
PizzaBox                     8
31                           3
88                           3
ExpansionChassis             2
127                          2
32      

In [18]:
## PowerPlatformRoleName

df_train.PowerPlatformRoleName.value_counts()

Mobile               1404248
Desktop               469919
Slate                 111676
Workstation            25131
SOHOServer              8585
UNKNOWN                 4667
EnterpriseServer        1600
AppliancePC              877
PerformanceServer         22
Unspecified                1
Name: PowerPlatformRoleName, dtype: int64

In [19]:
df_train.loc[(df_train.PowerPlatformRoleName=="UNKNOWN") | (df_train.PowerPlatformRoleName=="Unspecified"), "PowerPlatformRoleName"] = "Other"
df_train.PowerPlatformRoleName.value_counts()

Mobile               1404248
Desktop               469919
Slate                 111676
Workstation            25131
SOHOServer              8585
Other                   4668
EnterpriseServer        1600
AppliancePC              877
PerformanceServer         22
Name: PowerPlatformRoleName, dtype: int64

In [20]:
continuous_features_mean = ["Age"]
continuous_features_zero = ['ReadmissionCount'] + df_train.columns[8:].to_list()
categorical_features_other = ["OsBuildLab"]
categorical_features_none = ["PuaMode"]

In [21]:
##Summarizing Continous Variables
summarize(df_train[cont_variables])

'''
Some cateogrical identifiers have been read in as floats. 
We will add these to the list to use for categorical values and impute as "Other"

Dummies will be changed to categorical values and missing values coded as "Other"

Only a handful of variables are actually continuous: screen resolutions, 
processor core count, and the capacities for volume, ram, and disk. Impute missing calues with mean
'''

,Data Type,Missing Values,Unique Values,count,mean,std,min,25%,50%,75%,max
Def_IsBeta,int64,0,2,2026735.0,9.374684e-06,3.061797e-03,0.0,0.0,0.0,0.0,1.000000e+00
DefaultBrowsersIdentifier,float64,1928264,1101,98471.0,1.661578e+03,9.990188e+02,1.0,788.0,1632.0,2373.0,3.213000e+03
Def_Configuration,float64,8241,12367,2018494.0,4.785017e+04,1.401957e+04,6.0,49480.0,53447.0,53447.0,7.049800e+04
Has_Tpm_Chip,int64,0,2,2026735.0,9.879772e-01,1.089874e-01,0.0,1.0,1.0,1.0,1.000000e+00
CountryIdentifier,int64,0,222,2026735.0,1.080781e+02,6.303979e+01,1.0,51.0,97.0,162.0,2.220000e+02
CityIdentifier,float64,74094,67310,1952641.0,8.131843e+04,4.892447e+04,5.0,36825.0,82373.0,123959.0,1.679620e+05
OrganizationIdentifier,float64,625782,47,1400953.0,2.486884e+01,5.608307e+00,1.0,18.0,27.0,27.0,5.200000e+01
GeoNameIdentifier,float64,39,277,2026696.0,1.697618e+02,8.929723e+01,1.0,89.0,181.0,267.0,2.960000e+02
LocaleEnglishNameIdentifier,int64,0,264,2026735.0,1.228533e+02,6.932911e+01,1.0,75.0,88.0,182.0,2.830000e+02
OsBuild,int64,0,55,2026735.0,1.571973e+04,2.190748e+03,7600.0,15063.0,16299.0,17134.0,1.824200e+04


'\nSome cateogrical identifiers have been read in as floats. \nWe will add these to the list to use for categorical values and impute as "Other"\n\nDummies will be changed to categorical values and missing values coded as "Other"\n\nOnly a handful of variables are actually continuous: screen resolutions, \nprocessor core count, and the capacities for volume, ram, and disk. Impute missing calues with mean\n'

In [22]:
## Do the same cleaning process for the test data

df_test=pd.read_csv("test 3.csv")

## Remove NA columns

removals=["RtpStateBitfield","IsSxsPassiveMode","AVProductsInstalled", "AVProductsEnabled", "IeVerIdentifier",                     
                        "Census_OEMNameIdentifier","Census_OEMModelIdentifier","Census_ProcessorManufacturerIdentifier",
                        "Census_ProcessorModelIdentifier","Census_InternalBatteryType","Census_InternalBatteryNumberOfCharges",
                        "Census_OSInstallLanguageIdentifier","Census_OSUILocaleIdentifier","Census_IsFlightingInternal",
                        "Census_ThresholdOptIn","Census_FirmwareManufacturerIdentifier","Census_FirmwareVersionIdentifier",
                        "Census_IsWIMBootEnabled","Wdft_RegionIdentifier"]
len(removals) # There are 19 categories, so dataset should be 19 features smaller. 83-19=64
df_test.drop(removals, axis=1, inplace=True)
df_test.shape # Successfully dropped. One column less than df_train due to no response variable column

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(7853253, 63)

In [23]:
## Take out confusing variables for df_test

new_names={"ProductName":"Def_Name", "EngineVersion":"Def_EngineVersion", "AppVersion":"Def_AppVersion",
           "AvSigVersion":"Def_AvSigVersion","IsBeta":"Def_IsBeta", "AVProductStatesIdentifier":"Def_Configuration",
           "HasTpm":"Has_Tpm_Chip", "UacLuaenable":"AdminMode","Census_MDC2FormFactor":"Device_Type",
           "Census_DeviceFamily":"DeviceFamily","Census_ProcessorCoreCount":"ProcessorCoreCount",
           "Census_ProcessorClass":"ProcessorClass", "Census_PrimaryDiskTotalCapacity":"PrimaryDiskTotalCapacity",
           "Census_PrimaryDiskTypeName":"PrimaryDiskTypeName","Census_SystemVolumeTotalCapacity":"SystemVolumeTotalCapacity",
           "Census_HasOpticalDiskDrive":"HasOpticalDiskDrive"}
df_test.rename(columns=new_names,inplace=True)

In [24]:
## Loop for removing "Census" from those feature names in df_test

empty=list()
for column in df_test.columns:
    temp=column.replace("Census_","")
    empty.append(temp)
df_test.columns=empty
len(df_test.columns) ## Good - still has same number of features

63

In [25]:
## Final columns drops and checks before splitting data. Drop ProcessorClass and OSVer

df_train=df_train.drop(["ProcessorClass","OsVer"],axis=1)
df_test=df_test.drop(["ProcessorClass","OsVer"],axis=1)
#df_train.shape 62 columns
#df_test.shape 61 columns
## Looks good



In [26]:
df_train.shape

(2026735, 62)

In [27]:
## Drop the 1 df_training row that has NA for response variable

df_train[df_train.HasDetections.isnull()==True].index
df_train=df_train.drop([2026734,])
df_train.shape
## Good

(2026734, 62)

In [28]:
##Need to split data before running imputer on smaller_train
df_smaller_train, df_validation = train_test_split(df_train, test_size = 0.25, random_state = 201)


In [29]:
## Checking four datasets: df_train, df_test, df_smaller_train, df_validation

df_train.shape
df_test.shape
df_smaller_train.shape
df_validation.shape

## Looks good

(506684, 62)

In [30]:
##Splitting variables into lists for different imputers

continuous_features_mean = ['ProcessorCoreCount',
 'PrimaryDiskTotalCapacity',
 'SystemVolumeTotalCapacity',
 'TotalPhysicalRAM',
 'InternalPrimaryDiagonalDisplaySizeInInches',
 'InternalPrimaryDisplayResolutionHorizontal',
 'InternalPrimaryDisplayResolutionVertical']
categorical_features_other = ['MachineIdentifier', 
 'Def_Name',
 'Def_EngineVersion',
 'Def_AppVersion',
 'Def_AvSigVersion',
 'Def_IsBeta',
 'DefaultBrowsersIdentifier',
 'Def_Configuration',
 'Has_Tpm_Chip',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Platform',
 'Processor',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'AutoSampleOptIn',
 'SMode',
 'SmartScreen',
 'Firewall',
 'AdminMode',
 'Device_Type',
 'DeviceFamily',
 'PrimaryDiskTypeName', 
 'HasOpticalDiskDrive', 
 'ChassisTypeName',
 'PowerPlatformRoleName',
 'OSVersion',
 'OSArchitecture',
 'OSBranch',
 'OSBuildNumber',
 'OSBuildRevision',
 'OSEdition',
 'OSSkuName',
 'OSInstallTypeName',
 'OSWUAutoUpdateOptionsName',
 'IsPortableOperatingSystem',
 'GenuineStateName',
 'ActivationChannel',
 'IsFlightsDisabled',
 'FlightRing',
 'IsSecureBootEnabled',
 'IsVirtualDevice',
 'IsTouchEnabled',
 'IsPenCapable',
 'IsAlwaysOnAlwaysConnectedCapable',
 'Wdft_IsGamer',]
categorical_features_none = ["PuaMode"]

In [31]:
##Our three lists should have 61 variables. Since we have dropped "proccesorcore" and "OsVer", 
##and "HasDetections" is the target variable
len(continuous_features_mean) + len(categorical_features_other) + len(categorical_features_none)

61

#### The Dreaded Imputation Process

In [32]:
## Create the imputer classes. Don't need impute_zero class because not used.

continuous_imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
#continuous_imputer_zero = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
categorical_imputer_other = CategoricalImputer(other_threshold=0, fill_value="OTHER")
categorical_imputer_none = CategoricalImputer(other_threshold=0, fill_value="NONE")

In [33]:
## Impute for full df_train

continuous_imputer_mean.fit(df_train[continuous_features_mean])
#continuous_imputer_zero.fit(df_train[continuous_features_zero])
categorical_imputer_other.fit(df_train[categorical_features_other])
categorical_imputer_none.fit(df_train[categorical_features_none])

df_train.loc[:, continuous_features_mean] = continuous_imputer_mean.transform(df_train[continuous_features_mean])
#df_train.loc[:, continuous_features_zero] = continuous_imputer_zero.transform(df_train[continuous_features_zero])
df_train.loc[:, categorical_features_other] = categorical_imputer_other.transform(df_train[categorical_features_other])
df_train.loc[:, categorical_features_none] = categorical_imputer_none.transform(df_train[categorical_features_none])


In [34]:
## Check to make sure worked. 

summarize(df_train)
## Good

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
MachineIdentifier,object,0,2026734,2026734.0,2026734,0000028988387b115f69f31a3bf04f09,1,,,,,,,
Def_Name,object,0,6,2026734.0,6,win8defender,2005250,,,,,,,
Def_EngineVersion,object,0,64,2026734.0,64,1.1.15200.1,874115,,,,,,,
Def_AppVersion,object,0,99,2026734.0,99,4.18.1807.18075,1168237,,,,,,,
Def_AvSigVersion,object,0,7979,2026734.0,7979,1.273.1420.0,23406,,,,,,,
Def_IsBeta,object,0,2,2026734.0,2,0,2026715,,,,,,,
DefaultBrowsersIdentifier,object,0,1101,2026734.0,1101,OTHER,1928264,,,,,,,
Def_Configuration,object,0,12367,2026734.0,12367,53447.0,1323676,,,,,,,
Has_Tpm_Chip,object,0,2,2026734.0,2,1,2002367,,,,,,,
CountryIdentifier,object,0,222,2026734.0,222,43,90296,,,,,,,


In [35]:
## Impute for full df_test

df_test.loc[:, continuous_features_mean] = continuous_imputer_mean.transform(df_test[continuous_features_mean])
#df_train.loc[:, continuous_features_zero] = continuous_imputer_zero.transform(df_train[continuous_features_zero])
df_test.loc[:, categorical_features_other] = categorical_imputer_other.transform(df_test[categorical_features_other])
df_test.loc[:, categorical_features_none] = categorical_imputer_none.transform(df_test[categorical_features_none])


In [36]:
## Check to make sure worked. 

summarize(df_test)
## Good

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
MachineIdentifier,object,0,1,7853253.0,1,OTHER,7853253,,,,,,,
Def_Name,object,0,6,7853253.0,6,win8defender,7797245,,,,,,,
Def_EngineVersion,object,0,64,7853253.0,64,OTHER,3619785,,,,,,,
Def_AppVersion,object,0,99,7853253.0,99,4.18.1809.2,2738721,,,,,,,
Def_AvSigVersion,object,0,7890,7853253.0,7890,OTHER,6599804,,,,,,,
Def_IsBeta,object,0,2,7853253.0,2,0,7853207,,,,,,,
DefaultBrowsersIdentifier,object,0,824,7853253.0,824,OTHER,7566152,,,,,,,
Def_Configuration,object,0,7888,7853253.0,7888,53447.0,5508622,,,,,,,
Has_Tpm_Chip,object,0,2,7853253.0,2,1,7788201,,,,,,,
CountryIdentifier,object,0,222,7853253.0,222,43,457783,,,,,,,


In [37]:
## Write csv file of df_train_clean for tableau analysis

##df_train_cleaned=df_train
##df_train_cleaned.to_csv("df_train_cleaned.csv")

In [38]:
## Impute for df_smaller_train

df_smaller_train.loc[:, continuous_features_mean] = continuous_imputer_mean.transform(df_smaller_train[continuous_features_mean])
#df_smaller_train.loc[:, continuous_features_zero] = continuous_imputer_zero.transform(df_smaller_train[continuous_features_zero])
df_smaller_train.loc[:, categorical_features_other] = categorical_imputer_other.transform(df_smaller_train[categorical_features_other])
df_smaller_train.loc[:, categorical_features_none] = categorical_imputer_none.transform(df_smaller_train[categorical_features_none])


In [39]:
## Check to make sure it worked

summarize(df_smaller_train)
## Good

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
MachineIdentifier,object,0,1520050,1520050.0,1520050,30a4876e82a8a577118278ab0bff3d7a,1,,,,,,,
Def_Name,object,0,6,1520050.0,6,win8defender,1503977,,,,,,,
Def_EngineVersion,object,0,62,1520050.0,62,1.1.15200.1,655712,,,,,,,
Def_AppVersion,object,0,99,1520050.0,99,4.18.1807.18075,875929,,,,,,,
Def_AvSigVersion,object,0,7756,1520050.0,7756,1.273.1420.0,17474,,,,,,,
Def_IsBeta,object,0,2,1520050.0,2,0,1520036,,,,,,,
DefaultBrowsersIdentifier,object,0,955,1520050.0,955,OTHER,1446145,,,,,,,
Def_Configuration,object,0,10473,1520050.0,10473,53447.0,993115,,,,,,,
Has_Tpm_Chip,object,0,2,1520050.0,2,1,1501808,,,,,,,
CountryIdentifier,object,0,222,1520050.0,222,43,67274,,,,,,,


In [40]:
## Impute for df_validation

df_validation.loc[:, continuous_features_mean] = continuous_imputer_mean.transform(df_validation[continuous_features_mean])
#df_validation.loc[:, continuous_features_zero] = continuous_imputer_zero.transform(df_validation[continuous_features_zero])
df_validation.loc[:, categorical_features_other] = categorical_imputer_other.transform(df_validation[categorical_features_other])
df_validation.loc[:, categorical_features_none] = categorical_imputer_none.transform(df_validation[categorical_features_none])

In [41]:
## Check to make sure it worked

summarize(df_validation)
## Good

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
MachineIdentifier,object,0,506684,506684.0,506684,202f627ebcc990aad0e4156d0dc8957f,1,,,,,,,
Def_Name,object,0,4,506684.0,4,win8defender,501273,,,,,,,
Def_EngineVersion,object,0,54,506684.0,54,1.1.15200.1,218403,,,,,,,
Def_AppVersion,object,0,93,506684.0,93,4.18.1807.18075,292308,,,,,,,
Def_AvSigVersion,object,0,6464,506684.0,6464,1.273.1420.0,5932,,,,,,,
Def_IsBeta,object,0,2,506684.0,2,0,506679,,,,,,,
DefaultBrowsersIdentifier,object,0,590,506684.0,590,OTHER,482119,,,,,,,
Def_Configuration,object,0,5519,506684.0,5519,53447.0,330561,,,,,,,
Has_Tpm_Chip,object,0,2,506684.0,2,1,500559,,,,,,,
CountryIdentifier,object,0,222,506684.0,222,43,23022,,,,,,,


## It's Modeling Time LFGO

In [42]:
## Scale the continuous variables

scaler = StandardScaler()
scaler.fit(df_smaller_train[continuous_features_mean])
df_smaller_train[continuous_features_mean] = scaler.transform(df_smaller_train[continuous_features_mean])
df_validation[continuous_features_mean] = scaler.transform(df_validation[continuous_features_mean])

In [43]:
## Check if worked

summarize(df_smaller_train)
## Good

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
MachineIdentifier,object,0,1520050,1520050.0,1520050,30a4876e82a8a577118278ab0bff3d7a,1,,,,,,,
Def_Name,object,0,6,1520050.0,6,win8defender,1503977,,,,,,,
Def_EngineVersion,object,0,62,1520050.0,62,1.1.15200.1,655712,,,,,,,
Def_AppVersion,object,0,99,1520050.0,99,4.18.1807.18075,875929,,,,,,,
Def_AvSigVersion,object,0,7756,1520050.0,7756,1.273.1420.0,17474,,,,,,,
Def_IsBeta,object,0,2,1520050.0,2,0,1520036,,,,,,,
DefaultBrowsersIdentifier,object,0,955,1520050.0,955,OTHER,1446145,,,,,,,
Def_Configuration,object,0,10473,1520050.0,10473,53447.0,993115,,,,,,,
Has_Tpm_Chip,object,0,2,1520050.0,2,1,1501808,,,,,,,
CountryIdentifier,object,0,222,1520050.0,222,43,67274,,,,,,,


In [44]:
## Create matrices for tree training

X_train_tree = df_smaller_train[df_smaller_train.columns[1:-1]].copy() # Skipping MachineIdentifier
y_train_tree = df_smaller_train['HasDetections'].copy()
X_train_tree.columns

Index(['Def_Name', 'Def_EngineVersion', 'Def_AppVersion', 'Def_AvSigVersion',
       'Def_IsBeta', 'DefaultBrowsersIdentifier', 'Def_Configuration',
       'Has_Tpm_Chip', 'CountryIdentifier', 'CityIdentifier',
       'OrganizationIdentifier', 'GeoNameIdentifier',
       'LocaleEnglishNameIdentifier', 'Platform', 'Processor', 'OsBuild',
       'OsSuite', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition',
       'IsProtected', 'AutoSampleOptIn', 'PuaMode', 'SMode', 'SmartScreen',
       'Firewall', 'AdminMode', 'Device_Type', 'DeviceFamily',
       'ProcessorCoreCount', 'PrimaryDiskTotalCapacity', 'PrimaryDiskTypeName',
       'SystemVolumeTotalCapacity', 'HasOpticalDiskDrive', 'TotalPhysicalRAM',
       'ChassisTypeName', 'InternalPrimaryDiagonalDisplaySizeInInches',
       'InternalPrimaryDisplayResolutionHorizontal',
       'InternalPrimaryDisplayResolutionVertical', 'PowerPlatformRoleName',
       'OSVersion', 'OSArchitecture', 'OSBranch', 'OSBuildNumber',
       'OSBuildRevision',

In [45]:
y_train_tree

1694778    1.0
1109704    1.0
1065278    0.0
1268074    1.0
178044     0.0
          ... 
942948     1.0
1275233    0.0
1383936    0.0
1033769    0.0
580096     1.0
Name: HasDetections, Length: 1520050, dtype: float64

In [46]:
X_train_tree

,Def_Name,Def_EngineVersion,Def_AppVersion,Def_AvSigVersion,Def_IsBeta,DefaultBrowsersIdentifier,Def_Configuration,Has_Tpm_Chip,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,AdminMode,Device_Type,DeviceFamily,ProcessorCoreCount,PrimaryDiskTotalCapacity,PrimaryDiskTypeName,SystemVolumeTotalCapacity,HasOpticalDiskDrive,TotalPhysicalRAM,ChassisTypeName,InternalPrimaryDiagonalDisplaySizeInInches,InternalPrimaryDisplayResolutionHorizontal,InternalPrimaryDisplayResolutionVertical,PowerPlatformRoleName,OSVersion,OSArchitecture,OSBranch,OSBuildNumber,OSBuildRevision,OSEdition,OSSkuName,OSInstallTypeName,OSWUAutoUpdateOptionsName,IsPortableOperatingSystem,GenuineStateName,ActivationChannel,IsFlightsDisabled,FlightRing,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer
1694778,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1244.0,0,OTHER,53447.0,1,164,120983.0,27.0,205.0,172,windows10,x64,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NONE,0.0,OTHER,1.0,1.0,Desktop,Windows.Desktop,1.938984,-0.000890,SSD,-0.806817,0,0.408811,Desktop,1.244233,1.013547,0.851555,Desktop,10.0.17134.285,amd64,rs4_release,17134,285,Core,CORE,Reset,FullAuto,0,IS_GENUINE,OEM:NONSLP,0.0,Retail,0.0,0.0,0.0,0.0,0.0,0.0
1109704,win8defender,1.1.15200.1,4.18.1807.18075,1.275.769.0,0,OTHER,53447.0,1,211,OTHER,27.0,29.0,215,windows10,x64,15063,256,rs2,15063.0.amd64fre.rs2_release.170317-1834,Pro,1.0,0,NONE,0.0,OTHER,1.0,1.0,Desktop,Windows.Desktop,0.005753,-0.000821,Other,-0.646115,OTHER,-0.396487,Desktop,0.377715,-0.294151,0.010341,Desktop,10.0.15063.1058,amd64,rs2_release,15063,1058,Professional,PROFESSIONAL,IBSClean,UNKNOWN,0,IS_GENUINE,Retail,0.0,Retail,0.0,0.0,0.0,0.0,0.0,1.0
1065278,win8defender,1.1.15100.1,4.18.1807.18075,1.273.76.0,0,OTHER,7945.0,1,195,OTHER,18.0,276.0,74,windows10,x64,16299,768,rs3,16299.431.amd64fre.rs3_release_svc_escrow.1805...,Home,1.0,0,NONE,0.0,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,0.005753,-0.000821,HDD,0.249400,0,-0.396487,Notebook,-0.199963,-0.495755,-0.606548,Mobile,10.0.16299.431,amd64,rs3_release_svc_escrow,16299,431,Core,CORE,UUPUpgrade,Notify,0,IS_GENUINE,Retail,0.0,Retail,0.0,0.0,0.0,0.0,0.0,0.0
1268074,win8defender,1.1.15200.1,4.13.17134.228,1.275.948.0,0,OTHER,53447.0,1,155,82373.0,18.0,201.0,231,windows10,x64,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NONE,0.0,ExistsNotSet,1.0,1.0,Desktop,Windows.Desktop,0.005753,-0.000191,HDD,1.904127,0,0.408811,Desktop,1.074327,-1.427490,-0.606548,Workstation,10.0.17134.191,amd64,rs4_release,17134,191,Professional,PROFESSIONAL,IBSClean,FullAuto,0,INVALID_LICENSE,Retail,0.0,Retail,0.0,0.0,0.0,0.0,0.0,OTHER
178044,win8defender,1.1.15200.1,4.18.1807.18075,1.275.903.0,0,OTHER,53447.0,1,53,73162.0,27.0,71.0,52,windows10,x64,16299,256,rs3,16299.15.amd64fre.rs3_release.170928-1534,Pro,1.0,0,NONE,0.0,OTHER,1.0,1.0,Notebook,Windows.Desktop,0.005753,-0.000889,SSD,-0.785711,0,-0.396487,Notebook,-0.709680,-0.495755,-0.606548,Mobile,10.0.16299.371,amd64,rs3_release,16299,371,ProfessionalEducation,PROFESSIONAL,Upgrade,UNKNOWN,0,IS_GENUINE,Retail,0.0,Retail,OTHER,0.0,OTHER,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942948,win8defender,1.1.15100.1,4.14.17639.18041,1.273.975.0,0,OTHER,41571.0,1,84,62396.0,18.0,115.0,44,windows10,x64,16299,768,rs3,16299.15.amd64fre.rs3_release.170928-1534,Home,1.0,0,NONE,0.0,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,0.005753,-0.000731,HDD,0.278912,0,0.408811,Notebook,-0.199963,-0.495755,-0.606548,Mobile,10.0.16299.125,amd64,rs3_release,16299,125,Core,CORE,Upgrade

In [47]:
## Reclassifying cont/cat variables - removed MachineIdentifier

continuous_features_mean = ['ProcessorCoreCount',
 'PrimaryDiskTotalCapacity',
 'SystemVolumeTotalCapacity',
 'TotalPhysicalRAM',
 'InternalPrimaryDiagonalDisplaySizeInInches',
 'InternalPrimaryDisplayResolutionHorizontal',
 'InternalPrimaryDisplayResolutionVertical']
categorical_features_other = ['Def_Name',
 'Def_EngineVersion',
 'Def_AppVersion',
 'Def_AvSigVersion',
 'Def_IsBeta',
 'DefaultBrowsersIdentifier',
 'Def_Configuration',
 'Has_Tpm_Chip',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Platform',
 'Processor',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'AutoSampleOptIn',
 'SMode',
 'SmartScreen',
 'Firewall',
 'AdminMode',
 'Device_Type',
 'DeviceFamily',
 'PrimaryDiskTypeName', 
 'HasOpticalDiskDrive', 
 'ChassisTypeName',
 'PowerPlatformRoleName',
 'OSVersion',
 'OSArchitecture',
 'OSBranch',
 'OSBuildNumber',
 'OSBuildRevision',
 'OSEdition',
 'OSSkuName',
 'OSInstallTypeName',
 'OSWUAutoUpdateOptionsName',
 'IsPortableOperatingSystem',
 'GenuineStateName',
 'ActivationChannel',
 'IsFlightsDisabled',
 'FlightRing',
 'IsSecureBootEnabled',
 'IsVirtualDevice',
 'IsTouchEnabled',
 'IsPenCapable',
 'IsAlwaysOnAlwaysConnectedCapable',
 'Wdft_IsGamer',]
categorical_features_none = ["PuaMode"]
len(categorical_features_other + categorical_features_none)

53

In [48]:
len(categorical_features_other + categorical_features_none + continuous_features_mean)

60

In [49]:
categorical_variables = categorical_features_other + categorical_features_none
ordinal_encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)
categorical_variables

['Def_Name',
 'Def_EngineVersion',
 'Def_AppVersion',
 'Def_AvSigVersion',
 'Def_IsBeta',
 'DefaultBrowsersIdentifier',
 'Def_Configuration',
 'Has_Tpm_Chip',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Platform',
 'Processor',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'AutoSampleOptIn',
 'SMode',
 'SmartScreen',
 'Firewall',
 'AdminMode',
 'Device_Type',
 'DeviceFamily',
 'PrimaryDiskTypeName',
 'HasOpticalDiskDrive',
 'ChassisTypeName',
 'PowerPlatformRoleName',
 'OSVersion',
 'OSArchitecture',
 'OSBranch',
 'OSBuildNumber',
 'OSBuildRevision',
 'OSEdition',
 'OSSkuName',
 'OSInstallTypeName',
 'OSWUAutoUpdateOptionsName',
 'IsPortableOperatingSystem',
 'GenuineStateName',
 'ActivationChannel',
 'IsFlightsDisabled',
 'FlightRing',
 'IsSecureBootEnabled',
 'IsVirtualDevice',
 'IsTouchEnabled',
 'IsPenCapable',
 'IsAlwaysOnAlwaysConnectedCapable',
 'Wdft

In [50]:
ordinal_encoder.fit(X_train_tree[categorical_variables])
X_train_tree[categorical_variables] = ordinal_encoder.transform(X_train_tree[categorical_variables])

In [51]:
X_train_tree

,Def_Name,Def_EngineVersion,Def_AppVersion,Def_AvSigVersion,Def_IsBeta,DefaultBrowsersIdentifier,Def_Configuration,Has_Tpm_Chip,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,AdminMode,Device_Type,DeviceFamily,ProcessorCoreCount,PrimaryDiskTotalCapacity,PrimaryDiskTypeName,SystemVolumeTotalCapacity,HasOpticalDiskDrive,TotalPhysicalRAM,ChassisTypeName,InternalPrimaryDiagonalDisplaySizeInInches,InternalPrimaryDisplayResolutionHorizontal,InternalPrimaryDisplayResolutionVertical,PowerPlatformRoleName,OSVersion,OSArchitecture,OSBranch,OSBuildNumber,OSBuildRevision,OSEdition,OSSkuName,OSInstallTypeName,OSWUAutoUpdateOptionsName,IsPortableOperatingSystem,GenuineStateName,ActivationChannel,IsFlightsDisabled,FlightRing,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer
1694778,5.0,59.0,53.0,7282.0,0.0,954.0,6975.0,0.0,72.0,8544.0,15.0,110.0,75.0,1.0,1.0,6.0,7.0,4.0,239.0,4.0,1.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,1.0,1.938984,-0.000890,3.0,-0.806817,0.0,0.408811,14.0,1.244233,1.013547,0.851555,1.0,280.0,1.0,7.0,41.0,158.0,2.0,2.0,5.0,1.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1109704,5.0,59.0,53.0,7644.0,0.0,954.0,6975.0,0.0,125.0,60241.0,15.0,195.0,116.0,1.0,1.0,3.0,2.0,2.0,224.0,7.0,1.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,1.0,0.005753,-0.000821,2.0,-0.646115,1.0,-0.396487,14.0,0.377715,-0.294151,0.010341,1.0,186.0,1.0,3.0,27.0,15.0,13.0,14.0,1.0,5.0,0.0,1.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0
1065278,5.0,58.0,53.0,7191.0,0.0,954.0,10101.0,0.0,106.0,60241.0,6.0,183.0,229.0,1.0,1.0,4.0,7.0,3.0,232.0,4.0,1.0,0.0,0.0,0.0,9.0,1.0,1.0,7.0,1.0,0.005753,-0.000821,0.0,0.249400,0.0,-0.396487,26.0,-0.199963,-0.495755,-0.606548,3.0,248.0,1.0,5.0,34.0,178.0,2.0,2.0,6.0,2.0,0.0,1.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1268074,5.0,59.0,27.0,7716.0,0.0,954.0,6975.0,0.0,62.0,53089.0,6.0,106.0,134.0,1.0,1.0,6.0,2.0,4.0,239.0,7.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,1.0,0.005753,-0.000191,0.0,1.904127,0.0,0.408811,14.0,1.074327,-1.427490,-0.606548,9.0,277.0,1.0,7.0,41.0,118.0,13.0,14.0,1.0,1.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0
178044,5.0,59.0,53.0,7695.0,0.0,954.0,6975.0,0.0,170.0,49544.0,15.0,246.0,206.0,1.0,1.0,4.0,2.0,3.0,228.0,7.0,1.0,0.0,0.0,0.0,4.0,1.0,1.0,7.0,1.0,0.005753,-0.000889,3.0,-0.785711,0.0,-0.396487,26.0,-0.709680,-0.495755,-0.606548,3.0,246.0,1.0,4.0,34.0,173.0,15.0,14.0,8.0,5.0,0.0,1.0,3.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942948,5.0,58.0,34.0,7233.0,0.0,954.0,4648.0,0.0,204.0,45031.0,6.0,17.0,197.0,1.0,1.0,4.0,7.0,3.0,228.0,4.0,1.0,0.0,0.0,0.0,9.0,1.0,1.0,7.0,1.0,0.005753,-0.000731,0.0,0.278912,0.0,0.408811,26.0,-0.199963,-0.495755,-0.606548,3.0,235.0,1.0,4.0,34.0,34.0,2.0,2.0,8.0,2.0,0.0,1.0,3.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0
1275233,5.0,58.0,47.0,7126.0,0.0,954.0,6975.0,0.0,114.0,46627.0,15.0,173.0,153.0,1.0,2.0,4.0,2.0,3.0,230.0,7.0,1.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,1.0,-0.960862,-0.000890,2.0,-0.806624,0.0,-0.799136,14.0,0.309753,-0.495755,-0.606548,1.0,254.0,2.0,4.0,34.0,198.0,13.0,14.0,1.0,5.0,0.0,1.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1383936,5.0,59.0,53.0,7636.0,0.0,954.0,6975.0,0.0,48.0,50284.0,15.0,20.0,218.0,1.0,1.0,4.0,7.0,3.0,228.0,4.0,1.0,0.0,0.0,0.0,9.0,1.0,1.0,7.0,1.0,-0.960862,-0.000821,2.0,-0.487872,0.0,-0.799136,28.0,-0.199963,-0.495755,-0.606548,5.0,235.0,1.0,4.0,34.0,34.0,5.0,5.0,7.0,1.0,0.0,3.0,3.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0
1033769,5.0,59.0,53.0,7277.0,0.0,954.0,549.0,0.0,55.0,37273.0,6.0,84.0,210.0,1.0,2.0,2.0,7.0,1.0,176.0,4.0,1.0,0.0,0.0,0.0,9.0,1.0,1.0,3.0,1.0,0.005753,-0.000906,3.0,-1.067574,0.0,-0.799136,15.0,-1.11745

In [76]:
rf_model = RandomForestClassifier(n_estimators=500, max_features=15, min_samples_leaf=50, random_state=201, n_jobs=num_cpus)

In [ ]:
rf_model.fit(X_train_tree, y_train_tree)

In [ ]:
pd.DataFrame({'Importance': rf_model.feature_importances_}, index=X_train_tree.columns).sort_values(['Importance'], ascending=False)

In [ ]:
X_validation_tree = df_validation[df_smaller_train.columns[1:-1]].copy()
y_validation_tree = df_validation['HasDetections'].copy()

In [ ]:
X_validation_tree

In [ ]:
y_validation_tree

In [ ]:
X_validation_tree[categorical_variables] = ordinal_encoder.transform(X_validation_tree[categorical_variables])

In [ ]:
rf_pred = rf_model.predict_proba(X_validation_tree)[:,1]
rf_pred

In [ ]:
roc_auc_score(df_validation['HasDetections'], rf_pred)

In [61]:
##Training full model and testing on test data
## Scale the continuous variables

scaler = StandardScaler()
scaler.fit(df_train[continuous_features_mean])
df_train[continuous_features_mean] = scaler.transform(df_train[continuous_features_mean])
df_test[continuous_features_mean] = scaler.transform(df_test[continuous_features_mean])

In [62]:
X_train_tree_full = df_train[df_train.columns[1:-1]].copy() # Skipping MachineIdentifier
y_train_tree_full = df_train['HasDetections'].copy()

In [63]:
ordinal_encoder.fit(X_train_tree_full[categorical_variables])
X_train_tree_full[categorical_variables] = ordinal_encoder.transform(X_train_tree_full[categorical_variables])

In [64]:
rf_model_full = RandomForestClassifier(n_estimators=100, max_features=12, min_samples_leaf=50, random_state=201, n_jobs=num_cpus)

In [65]:
rf_model_full.fit(X_train_tree_full, y_train_tree_full)

RandomForestClassifier(max_features=12, min_samples_leaf=50, n_jobs=16,
                       random_state=201)

In [66]:
pd.DataFrame({'Importance': rf_model_full.feature_importances_}, index=X_train_tree_full.columns).sort_values(['Importance'], ascending=False)

,Importance
SmartScreen,0.261161
Def_Configuration,0.144079
Def_AvSigVersion,0.074048
Def_EngineVersion,0.039793
SystemVolumeTotalCapacity,0.038411
CityIdentifier,0.030260
InternalPrimaryDiagonalDisplaySizeInInches,0.029461
Def_AppVersion,0.029302
CountryIdentifier,0.027572
LocaleEnglishNameIdentifier,0.023682


In [67]:
X_test_tree = df_test[df_train.columns[1:-1]].copy()
X_test_tree

,Def_Name,Def_EngineVersion,Def_AppVersion,Def_AvSigVersion,Def_IsBeta,DefaultBrowsersIdentifier,Def_Configuration,Has_Tpm_Chip,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,AdminMode,Device_Type,DeviceFamily,ProcessorCoreCount,PrimaryDiskTotalCapacity,PrimaryDiskTypeName,SystemVolumeTotalCapacity,HasOpticalDiskDrive,TotalPhysicalRAM,ChassisTypeName,InternalPrimaryDiagonalDisplaySizeInInches,InternalPrimaryDisplayResolutionHorizontal,InternalPrimaryDisplayResolutionVertical,PowerPlatformRoleName,OSVersion,OSArchitecture,OSBranch,OSBuildNumber,OSBuildRevision,OSEdition,OSSkuName,OSInstallTypeName,OSWUAutoUpdateOptionsName,IsPortableOperatingSystem,GenuineStateName,ActivationChannel,IsFlightsDisabled,FlightRing,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer
0,win8defender,OTHER,OTHER,OTHER,0,OTHER,53447.0,1,43,58552.0,18.0,53.0,42,windows10,x64,15063,768,rs2,15063.0.amd64fre.rs2_release.170317-1834,Home,1.0,0,NONE,OTHER,OTHER,1.0,1.0,Notebook,Windows.Desktop,0.004973,-0.000712,SSD,-0.778780,0,0.401982,Notebook,-0.200370,1.012323,0.850826,Mobile,OTHER,amd64,rs2_release,15063,OTHER,Core,CORE,Reset,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,OEM:DM,0.0,Retail,OTHER,0.0,0,0,0.0,0.0
1,win8defender,OTHER,4.18.1809.2,OTHER,0,OTHER,53447.0,1,68,71395.0,OTHER,276.0,74,windows10,x64,16299,768,rs3,16299.431.amd64fre.rs3_release_svc_escrow.1805...,Home,1.0,0,NONE,OTHER,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,0.004973,-0.000402,HDD,4.612392,0,0.401982,Notebook,-0.200370,-0.495316,-0.606198,Mobile,10.0.16299.611,amd64,rs3_release_svc_escrow,16299,611,Core,CORE,UUPUpgrade,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,Retail,0.0,Retail,OTHER,0.0,0,0,0.0,1.0
2,win8defender,1.1.15300.6,4.18.1809.2,OTHER,0,OTHER,49480.0,1,201,66202.0,OTHER,267.0,251,windows10,x64,14393,768,rs1,14393.2189.amd64fre.rs1_release.180329-1711,Home,1.0,0,NONE,OTHER,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,-0.955826,-0.000812,SSD,-1.068388,0,-0.391768,Notebook,-0.472471,-0.729354,-0.830355,Mobile,10.0.14393.2189,amd64,rs1_release,14393,2189,CoreSingleLanguage,CORE_SINGLELANGUAGE,Other,UNKNOWN,0,IS_GENUINE,OEM:DM,0.0,Retail,OTHER,0.0,0,0,0.0,1.0
3,win8defender,OTHER,OTHER,OTHER,0,OTHER,42160.0,1,29,120917.0,OTHER,35.0,171,windows10,x64,16299,768,rs3,16299.15.amd64fre.rs3_release.170928-1534,Home,1.0,0,NONE,OTHER,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,-0.955826,-0.000714,HDD,0.190144,0,-0.391768,Notebook,-0.455464,-0.495316,-0.606198,Mobile,10.0.16299.371,amd64,rs3_release,16299,371,CoreSingleLanguage,CORE_SINGLELANGUAGE,Upgrade,Notify,0,IS_GENUINE,OEM:DM,0.0,Retail,OTHER,0.0,0,0,0.0,0.0
4,win8defender,OTHER,4.18.1809.2,OTHER,0,OTHER,53447.0,1,171,124736.0,18.0,211.0,182,windows10,x64,16299,768,rs3,16299.15.amd64fre.rs3_release.170928-1534,Home,1.0,0,NONE,OTHER,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,0.004973,-0.000714,HDD,0.258059,OTHER,-0.788642,Notebook,-0.200370,-0.495316,-0.606198,Mobile,10.0.16299.371,amd64,rs3_release,16299,371,CoreSingleLanguage,CORE_SINGLELANGUAGE,Update,UNKNOWN,0,IS_GENUINE,Retail,0.0,Retail,OTHER,0.0,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7853248,win8defender,OTHER,OTHER,OTHER,0,OTHER,53447.0,1,29,157045.0,OTHER,35.0,171,windows10,x64,16299,768,rs3,16299.637.amd64fre.rs3_release_svc.180808-1748,Home,1.0,0,NONE,OTHER,OTHER,1.0,1.0,Notebook,Windows.Desktop,0.004973,-0.000714,HDD,0.305352,0,-0.391768,Notebook,-0.472471,-0.495316,-0.606198,Mobile,10.0.16299.15,amd64,rs3_release,16299,15,CoreSingleLanguage,CORE_SINGLELANGUAGE,IBSClean,UNKNOWN,0,IS_

In [68]:
X_test_tree[categorical_variables] = ordinal_encoder.transform(X_test_tree[categorical_variables])

In [69]:
rf_pred = rf_model.predict_proba(X_test_tree)[:,1]
rf_pred

array([0.34083553, 0.35473549, 0.33296994, ..., 0.42285927, 0.36986491,
       0.3276148 ])

In [70]:
sample_submission = pd.read_csv('sample_submission.csv')

In [71]:
sample_submission.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.5
1,00000176ac758d54827acd545b6315a5,0.5
2,0000019dcefc128c2d4387c1273dae1d,0.5
3,0000055553dc51b1295785415f1a224d,0.5
4,00000574cefffeca83ec8adf9285b2bf,0.5


In [72]:
sample_submission['HasDetections'] = rf_pred

In [73]:
sample_submission.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.340836
1,00000176ac758d54827acd545b6315a5,0.354735
2,0000019dcefc128c2d4387c1273dae1d,0.332970
3,0000055553dc51b1295785415f1a224d,0.438632
4,00000574cefffeca83ec8adf9285b2bf,0.384977


In [74]:
sample_submission.to_csv('finalSubmission.csv', index=False)

In [75]:
sample_submission.shape

(7853253, 2)